<a href="https://colab.research.google.com/github/awwnchal/New-Feature-Performance-evaluation/blob/main/Introducing_an_online_community_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#  1

In [ ]:
did=pd.read_excel('/content/drive/MyDrive/Data.xlsx',
                  'Data 1')

In [ ]:
did.head()

,Customer ID,Month Before,Month After,Joined?,difference
0,1000201,58,132,1,74
1,1000202,135,154,0,19
2,1000203,35,68,0,33
3,1000204,95,170,1,75
4,1000205,91,149,1,58


In [ ]:
did=did[["Month Before","Month After", "Joined?"]]

In [ ]:
did.groupby("Joined?").mean()

,Month Before,Month After
Joined?,,
0,70.376068,101.247863
1,88.134146,148.024390


## Manual Way

In [ ]:
mean_rev_not_before=did.groupby('Joined?').mean().iloc[0, 0]
mean_rev_not_after=did.groupby('Joined?').mean().iloc[0, 1]
mean_rev_yes_before=did.groupby('Joined?').mean().iloc[1, 0]
mean_rev_yes_after=did.groupby('Joined?').mean().iloc[1, 1]

not_diff=mean_rev_not_after-mean_rev_not_before
yes_diff=mean_rev_yes_after-mean_rev_yes_before

yes_diff-not_diff

29.01844903064415

## Regression Way

In [ ]:
df_before=did[["Month Before","Joined?"]]
df_before['t']=0
df_before.columns=[["total_rev","g","t"]]

df_after=did[["Month After", "Joined?"]]
df_after['t']=1
df_after.columns=[["total_rev", "g", "t"]]

df_did=pd.concat([df_before, df_after])

In [ ]:
df_did.isna().sum()

total_rev    0
g            0
t            0
dtype: int64

In [ ]:
df_did['gt']=df_did['t'].values*df_did['g'].values

In [ ]:
df_did.dtypes

total_rev    int64
g            int64
t            int64
gt           int64
dtype: object

In [ ]:
from statsmodels.formula.api import ols
ols = ols('total_rev ~ g + t + gt', data=df_did).fit()
print(ols.summary())
## Note that the results are the same and that B3 is significant at any confidence level.

                            OLS Regression Results                            
Dep. Variable:              total_rev   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.336
Method:                 Least Squares   F-statistic:                     67.89
Date:                Wed, 09 Nov 2022   Prob (F-statistic):           2.09e-35
Time:                        02:40:05   Log-Likelihood:                -2016.6
No. Observations:                 398   AIC:                             4041.
Df Residuals:                     394   BIC:                             4057.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     70.3761      3.567     19.729      0.0

# 2

In [ ]:
churn_df=pd.read_excel('/content/drive/MyDrive/Data.xlsx', 'Data 2')

In [ ]:
churn_df.head()

,Customer ID,Joined?,Customer Age with Firm at time of launching the online community,Churned at 3 months after launch of the online community,Average Spend Last 3 months of Life with the firm
0,1000201,1,7.008895,0,88.000000
1,1000202,0,6.595568,1,102.666667
2,1000203,0,7.932284,1,45.333333
3,1000204,1,2.391398,1,113.333333
4,1000205,1,5.335660,0,99.333333


In [ ]:
churn_df['Average Spend Last 3 months of Life with the firm']=np.round(
    churn_df['Average Spend Last 3 months of Life with the firm'])

In [ ]:
import statsmodels.api as sm
x=churn_df[["Joined?",
            "Customer Age with Firm at time of launching the online community",
            "Average Spend Last 3 months of Life with the firm"]]
y=churn_df[' Churned at 3 months after launch of the online community']
x=sm.add_constant(x)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
len(churn_df[(churn_df[' Churned at 3 months after launch of the online community']==1) &
         (churn_df['Joined?']==1)])/sum(churn_df['Joined?']==1)

0.7073170731707317

In [ ]:
len(churn_df[(churn_df[' Churned at 3 months after launch of the online community']==1) &
         (churn_df['Joined?']==0)])/sum(churn_df['Joined?']==0)

0.5128205128205128

In [ ]:
log_churn=sm.Logit(y, x).fit()
log_churn.summary()

Optimization terminated successfully.
         Current function value: 0.654587
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                                               Logit Regression Results                                              
=====================================================================================================================
Dep. Variable:      Churned at 3 months after launch of the online community   No. Observations:                  199
Model:                                                                 Logit   Df Residuals:                      195
Method:                                                                  MLE   Df Model:                            3
Date:                                                       Fri, 11 Nov 2022   Pseudo R-squ.:                 0.03133
Time:                                                               07:19:30   Log-Likelihood:                -130.26
converged:                                                              True   LL-Null:                       -134.48
Covariance Type:                                                   nonrobust   LLR p-value:                   0.03796
====================================================================================================================================
                                                                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
const                                                                0.4672      0.536      0.872      0.383      -0.583       1.517
Joined?                                                              0.9196      0.355      2.589      0.010       0.223       1.616
Customer Age with Firm at time of launching the online community    -0.0518      0.073     -0.709      0.479      -0.195       0.092
Average Spend Last 3 months of Life with the firm                   -0.0030      0.006     -0.524      0.600      -0.014       0.008
====================================================================================================================================
"""

In [ ]:
params=log_churn.params
ci=log_churn.conf_int()
ci['Odds Ratio'] = params
ci.columns = ['5%', '95%', 'Odds Ratio']
print(np.exp(ci))

                                                          5%       95%  \
const                                               0.558242  4.560641   
Joined?                                             1.250391  5.031874   
Customer Age with Firm at time of launching the...  0.822677  1.095848   
Average Spend Last 3 months of Life with the firm   0.986036  1.008163   

                                                    Odds Ratio  
const                                                 1.595600  
Joined?                                               2.508348  
Customer Age with Firm at time of launching the...    0.949489  
Average Spend Last 3 months of Life with the firm     0.997038  


odds ratio interpretation: If someone joins the community they are 150.9% more likely to churn than someone who has not joined the community.

# 3

In [ ]:
churn_df_q3=pd.read_csv("/content/drive/MyDrive/Assignment3/Assignment3HW3_Data.csv")
#df_question_1=pd.read_excel('/content/drive/MyDrive/Assignment3/Assignment3HW3_Data.xlsx',
                  #'Data 1')

In [ ]:
#churn_df_q3=churn_df_q3.merge(did[['Customer ID',"Month Before","Month After"]], on='Customer ID')

In [ ]:
i=0.05
churn_df_q3['m']=0.5*3*churn_df_q3['Average Spend Last 3 months of Life with the firm']
#churn_df_q3['m_before']=0.5*churn_df_q3['Month Before']
#churn_df_q3['m_after']=0.5*churn_df_q3['Month After']

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.round(log_churn.predict()),churn_df_q3[' Churned at 3 months after launch of the online community'])

0.6130653266331658

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(np.round(log_churn.predict()),churn_df_q3[' Churned at 3 months after launch of the online community'])

0.5888563049853373

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
confusion_matrix(np.round(log_churn.predict()),churn_df_q3[' Churned at 3 months after launch of the online community'])

array([[24, 20],
       [57, 98]])

In [ ]:
churn_df_q3["r"]=1-log_churn.predict()
#sum(churn_df_q3[' Churned at 3 months after launch of the online community']==0)/len(churn_df_q3)
#log_churn.predict(x)

In [ ]:
churn_df_q3['clv']=churn_df_q3['m']*((1+i)/(1+i-churn_df_q3["r"]))
#churn_df_q3['clv_before']=churn_df_q3['m_before']*((1+i)/(1+i+churn_df_q3["r"]))
#churn_df_q3['clv_after']=churn_df_q3['m_after']*((1+i)/(1+i+churn_df_q3["r"]))
#(churn_df_q3['m']*churn_df_q3['r'])/(1+i)

In [ ]:
churn_df_q3

,Customer ID,Joined?,Customer Age with Firm at time of launching the online community,Churned at 3 months after launch of the online community,Average Spend Last 3 months of Life with the firm,m,r,clv
0,1000201,1,7,0,88,132.0,0.318086,189.366553
1,1000202,0,7,1,103,154.5,0.544906,321.178098
2,1000203,0,8,1,45,67.5,0.519331,133.557890
3,1000204,1,2,1,113,169.5,0.283379,232.155202
4,1000205,1,5,0,99,148.5,0.306469,209.708743
...,...,...,...,...,...,...,...,...
194,1000395,0,4,1,95,142.5,0.501258,272.669265
195,1000396,0,5,1,28,42.0,0.472365,76.345737
196,1000397,0,5,0,25,37.5,0.464734,67.277096
197,1000398,1,2,1,95,142.5,0.265905,190.825167


In [ ]:
churn_df_q3.groupby('Joined?')[['clv']].mean()

,clv
Joined?,
0,191.773491
1,206.660052


In [ ]:
churn_df_q3.columns

Index(['Customer ID', 'Joined?',
       'Customer Age with Firm at time of launching the online community',
       ' Churned at 3 months after launch of the online community',
       'Average Spend Last 3 months of Life with the firm', 'm', 'r', 'clv'],
      dtype='object')

In [ ]:
churn_df_q3.corr()

,Customer ID,Joined?,Customer Age with Firm at time of launching the online community,Churned at 3 months after launch of the online community,Average Spend Last 3 months of Life with the firm,m,r,clv
Customer ID,1.000000,-0.080153,-0.063062,0.083331,-0.066226,-0.066226,0.055355,-0.044380
Joined?,-0.080153,1.000000,-0.026346,0.194862,0.506416,0.506416,-0.956028,0.100803
Customer Age with Firm at time of launching the online community,-0.063062,-0.026346,1.000000,-0.058872,-0.019496,-0.019496,0.262070,0.085733
Churned at 3 months after launch of the online community,0.083331,0.194862,-0.058872,1.000000,0.069539,0.069539,-0.204219,-0.007501
Average Spend Last 3 months of Life with the firm,-0.066226,0.506416,-0.019496,0.069539,1.000000,1.000000,-0.341171,0.893906
m,-0.066226,0.506416,-0.019496,0.069539,1.000000,1.000000,-0.341171,0.893906
r,0.055355,-0.956028,0.262070,-0.204219,-0.341171,-0.341171,1.000000,0.092035
clv,-0.044380,0.100803,0.085733,-0.007501,0.893906,0.893906,0.092035,1.000000


In [ ]:
import statsmodels.api as sm
x=churn_df_q3[['Joined?',"Customer Age with Firm at time of launching the online community",
               ' Churned at 3 months after launch of the online community']]
y=churn_df_q3['clv']
x=sm.add_constant(x)
log_churn2=sm.OLS(y, x).fit()
log_churn2.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    clv   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.225
Date:                Fri, 11 Nov 2022   Prob (F-statistic):              0.302
Time:                        07:20:13   Log-Likelihood:                -1133.5
No. Observations:                 199   AIC:                             2275.
Df Residuals:                     195   BIC:                             2288.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
const                                                              180.6766     13.849     13.046      0.000     153.363     207.990
Joined?                                                             15.8976     10.683      1.488      0.138      -5.172      36.967
Customer Age with Firm at time of launching the online community     3.0852      2.515      1.227      0.221      -1.875       8.045
 Churned at 3 months after launch of the online community           -3.4540     10.718     -0.322      0.748     -24.593      17.685
==============================================================================
Omnibus:                       37.154   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                8.635
Skew:                           0.004   Prob(JB):                       0.0133
Kurtosis:                       1.980   Cond. No.                         14.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

coefficient interpretation: Participation in the community had no statistically significant affect on customer lifetime value

# 



In [ ]:
campaign=pd.read_excel("/content/drive/MyDrive/Assignment3/Assignment3HW3_Data.xlsx", 'Data 3')

In [ ]:
campaign['Average Spend Last 3 months of Life with the firm']=np.round(campaign['Average Spend Last 3 months of Life with the firm'])

In [ ]:
campaign.columns

Index(['Campaign/Organic', 'Customer ID', 'Month Before', 'Month After',
       'Customer Age with Firm at time of launching the online community',
       'Joined?', 'Churned at 3 months',
       'Average Spend Last 3 months of Life with the firm'],
      dtype='object')

In [ ]:
did2=campaign[["Month Before", "Month After","Campaign/Organic"]]

In [ ]:
did2.groupby("Campaign/Organic").mean()

,Month Before,Month After
Campaign/Organic,,
0,75.320000,120.453333
1,79.129032,120.564516


In [ ]:
did2=campaign[campaign["Campaign/Organic"]==1]

In [ ]:
did2.groupby("Campaign/Organic")[['Month Before','Month After']].mean()

,Month Before,Month After
Campaign/Organic,,
1,79.129032,120.564516


In [ ]:
import statsmodels.api as sm
df_before=did2[["Month Before","Joined?"]]
df_before['t']=0
df_before.columns=[["total_rev","g","t"]]

df_after=did2[["Month After", "Joined?"]]
df_after['t']=1
df_after.columns=[["total_rev", "g", "t"]]

df_did=pd.concat([df_before, df_after])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
df_did['gt']=df_did['t'].values*df_did['g'].values

In [ ]:
df_did.head()

,total_rev,g,t,gt
5,37,1,0,0
6,78,1,0,0
8,119,0,0,0
9,28,1,0,0
10,63,0,0,0


In [ ]:
x=df_did[['g','t','gt']]
y=df_did['total_rev']
x=sm.add_constant
from statsmodels.formula.api import ols
ols = ols('total_rev ~ g + t + gt', data=df_did).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:              total_rev   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     40.65
Date:                Wed, 09 Nov 2022   Prob (F-statistic):           2.44e-21
Time:                        05:34:54   Log-Likelihood:                -1255.1
No. Observations:                 248   AIC:                             2518.
Df Residuals:                     244   BIC:                             2532.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     71.5333      4.442     16.103      0.0

In [ ]:
did3=campaign[campaign["Campaign/Organic"]==0]
df_before=did3[["Month Before","Joined?"]]
df_before['t']=0
df_before.columns=[["total_rev","g","t"]]

df_after=did3[["Month After", "Joined?"]]
df_after['t']=1
df_after.columns=[["total_rev", "g", "t"]]

df_did=pd.concat([df_before, df_after])

df_did['gt']=df_did['t'].values*df_did['g'].values

x=df_did[['g','t','gt']]
y=df_did['total_rev']
x=sm.add_constant
from statsmodels.formula.api import ols
ols = ols('total_rev ~ g + t + gt', data=df_did).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:              total_rev   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     26.88
Date:                Wed, 09 Nov 2022   Prob (F-statistic):           6.70e-14
Time:                        05:34:59   Log-Likelihood:                -761.02
No. Observations:                 150   AIC:                             1530.
Df Residuals:                     146   BIC:                             1542.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     68.3095      6.045     11.300      0.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
x=campaign[["Customer Age with Firm at time of launching the online community",
            "Churned at 3 months",
            "Campaign/Organic"]]
y=campaign['Average Spend Last 3 months of Life with the firm']
x=sm.add_constant(x)
ols2=sm.OLS(y,x).fit()
ols2.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                            OLS Regression Results                                           
=============================================================================================================
Dep. Variable:     Average Spend Last 3 months of Life with the firm   R-squared:                       0.008
Model:                                                           OLS   Adj. R-squared:                 -0.007
Method:                                                Least Squares   F-statistic:                    0.5145
Date:                                               Wed, 09 Nov 2022   Prob (F-statistic):              0.673
Time:                                                       03:24:15   Log-Likelihood:                -960.33
No. Observations:                                                199   AIC:                             1929.
Df Residuals:                                                    195   BIC:                             1942.
Df Model:                                                          3                                         
Covariance Type:                                           nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
const                                                               81.2238      6.251     12.995      0.000      68.897      93.551
Customer Age with Firm at time of launching the online community    -0.8124      1.075     -0.756      0.451      -2.932       1.307
Churned at 3 months                                                  4.1558      4.406      0.943      0.347      -4.534      12.846
Campaign/Organic                                                    -0.0146      4.461     -0.003      0.997      -8.812       8.783
==============================================================================
Omnibus:                       14.012   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.001   Jarque-Bera (JB):                5.393
Skew:                          -0.029   Prob(JB):                       0.0674
Kurtosis:                       2.196   Cond. No.                         15.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
pip install pysal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 5.7 MB 48.5 MB/s 
     |████████████████████████████████| 147 kB 42.4 MB/s 
     |████████████████████████████████| 243 kB 46.0 MB/s 
     |████████████████████████████████| 46 kB 3.9 MB/s 
     |████████████████████████████████| 116 kB 41.9 MB/s 
     |████████████████████████████████| 60 kB 3.9 MB/s 
     |████████████████████████████████| 220 kB 44.4 MB/s 
     |████████████████████████████████| 84 kB 2.0 MB/s 
     |████████████████████████████████| 2.4 MB 41.6 MB/s 
     |████████████████████████████████| 41 kB 662 kB/s 
     |████████████████████████████████| 1.0 MB 14.0 MB/s 
     |████████████████████████████████| 179 kB 19.7 MB/s 
     |████████████████████████████████| 2.1 MB 20.0 MB/s 
     |████████████████████████████████| 16.7 MB 359 kB/s 
     |████████████████████████████████| 6

In [ ]:
from pysal.model import spreg
from pysal.lib import weights
from pysal.explore import esda

/usr/local/lib/python3.7/dist-packages/spaghetti/network.py:36: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(f"{dep_msg}", FutureWarning)
